In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import threading

class DataVisualizationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Dynamic Data Visualization Tool")
        self.root.geometry("900x750")

        # Styling with ttk
        style = ttk.Style()
        style.theme_use("clam")  # Use a modern theme

        # Create the main frame
        main_frame = ttk.Frame(root, padding="10 10 10 10")
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Title Label
        title_label = ttk.Label(main_frame, text="Data Visualization Tool", font=("Helvetica", 24))
        title_label.pack(pady=20)

        # Upload Button
        self.upload_button = ttk.Button(main_frame, text="Upload Dataset", command=self.upload_dataset)
        self.upload_button.pack(pady=10)

        # Plot Type Selection
        plot_type_frame = ttk.Frame(main_frame)
        plot_type_frame.pack(pady=10)
        
        plot_type_label = ttk.Label(plot_type_frame, text="Select Plot Type:", font=("Helvetica", 14))
        plot_type_label.grid(row=0, column=0, padx=10)

        self.plot_type = ttk.Combobox(plot_type_frame, values=["Scatter", "Line", "Bar", "Histogram", "Boxplot"], font=("Helvetica", 12))
        self.plot_type.grid(row=0, column=1, padx=10)
        self.plot_type.current(0)

        # X-axis Column Selection
        x_axis_frame = ttk.Frame(main_frame)
        x_axis_frame.pack(pady=10)

        column_x_label = ttk.Label(x_axis_frame, text="Select X-axis Column:", font=("Helvetica", 14))
        column_x_label.grid(row=0, column=0, padx=10)

        self.column_x = ttk.Combobox(x_axis_frame, values=[], font=("Helvetica", 12))
        self.column_x.grid(row=0, column=1, padx=10)

        # Y-axis Column Selection
        y_axis_frame = ttk.Frame(main_frame)
        y_axis_frame.pack(pady=10)

        column_y_label = ttk.Label(y_axis_frame, text="Select Y-axis Column:", font=("Helvetica", 14))
        column_y_label.grid(row=0, column=0, padx=10)

        self.column_y = ttk.Combobox(y_axis_frame, values=[], font=("Helvetica", 12))
        self.column_y.grid(row=0, column=1, padx=10)

        # Number of Data Points Selection
        data_points_frame = ttk.Frame(main_frame)
        data_points_frame.pack(pady=10)

        data_points_label = ttk.Label(data_points_frame, text="Number of Data Points:", font=("Helvetica", 14))
        data_points_label.grid(row=0, column=0, padx=10)

        self.data_points = tk.Entry(data_points_frame, font=("Helvetica", 12))
        self.data_points.grid(row=0, column=1, padx=10)
        self.data_points.insert(0, "20")  # Default value

        # Generate Plot Button
        self.plot_button = ttk.Button(main_frame, text="Generate Plot", command=self.run_plot_in_thread)
        self.plot_button.pack(pady=20)

        # Exit Button
        self.exit_button = ttk.Button(main_frame, text="Exit", command=root.quit)
        self.exit_button.pack(pady=10)

        # Initialize dataset variable
        self.dataset = None

    def upload_dataset(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            try:
                self.dataset = pd.read_csv(file_path)
                self.column_x['values'] = self.dataset.columns.tolist()
                self.column_y['values'] = self.dataset.columns.tolist()
                messagebox.showinfo("Success", "Dataset uploaded successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to upload dataset: {e}")

    def run_plot_in_thread(self):
        threading.Thread(target=self.generate_plot_popup).start()

    def generate_plot_popup(self):
        if self.dataset is not None:
            x_col = self.column_x.get()
            y_col = self.column_y.get()
            plot_type = self.plot_type.get().lower()

            try:
                n_points = int(self.data_points.get())  # Get the number of data points
            except ValueError:
                messagebox.showerror("Error", "Please enter a valid number for data points.")
                return

            if x_col and y_col and n_points > 0:
                try:
                    # Slice the dataset to use only the specified number of data points
                    data = self.dataset[[x_col, y_col]].head(n_points)

                    # Create a new window for the plot
                    plot_window = tk.Toplevel(self.root)
                    plot_window.title(f"{plot_type.capitalize()} Plot: {y_col} vs {x_col}")
                    plot_window.geometry("800x600")

                    # Use Matplotlib to generate the plot
                    plt.figure(figsize=(10, 6))
                    if plot_type == "scatter":
                        sns.scatterplot(data=data, x=x_col, y=y_col)
                    elif plot_type == "line":
                        sns.lineplot(data=data, x=x_col, y=y_col)
                    elif plot_type == "bar":
                        sns.barplot(data=data, x=x_col, y=y_col)
                    elif plot_type == "histogram":
                        sns.histplot(data=data[x_col], kde=True)
                    elif plot_type == "boxplot":
                        sns.boxplot(data=data, x=x_col, y=y_col)
                    else:
                        messagebox.showwarning("Warning", "Unsupported plot type selected.")
                        return

                    plt.title(f'{plot_type.capitalize()} plot of {y_col} vs {x_col}')
                    plt.xlabel(x_col)
                    plt.ylabel(y_col)

                    # Embed the plot in the popup window
                    canvas = FigureCanvasTkAgg(plt.gcf(), master=plot_window)
                    canvas.draw()
                    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

                except Exception as e:
                    messagebox.showerror("Error", f"Failed to generate plot: {e}")
            else:
                messagebox.showwarning("Warning", "Please select both X and Y axis columns and enter a valid number of data points.")
        else:
            messagebox.showwarning("Warning", "Please upload a dataset first.")

# Create the main window
root = tk.Tk()
app = DataVisualizationApp(root)
root.mainloop()
